# CS202: Compiler Construction

## In-class Exercises, Week of 02/07/2022

----

# Questions from Last Week

## Question 6

Describe the assign-homes pass.

- Follow the structure of x86 assembly program ASTs (ah_arg, ah_instr, ah_block)
- Maintain a global dictionary `homes` mapping variable names to stack locations
- When you encounter a variable
    - If it already exists in `homes`, just return the location `homes` gives for it
    - Else: create a new stack location
        - Create a new AST node for the location
            - Register: `rbp`
            - Offset: `-(len(homes) * 8) + 8`
        - Set `homes[x] = x86.Deref('rbp', offset)`
        - Return `homes[x]`
- At the end, align the stack to 16 bytes and return stack size along with program

----
# Patch-instructions pass

The patch-instructions pass fixes instructions with two in-memory arguments, by using the `rax` register as a temporary location.

## Question 7

What is wrong with the following instructions?

```
movq -8(%rbp), -16(%rbp)
addq -24(%rbp), -16(%rbp)
```

In both instructions, both args are memory locations. In x86 assembly, only one arg is allowed to be a memory location.

## Question 8

Fix the instructions above.

Use %rax as a temporary location
```
movq -8(%rbp), %rax
movq %rax, -16(%rbp)
movq -24(%rbp), %rax
movq %rax, -16(%rbp)
```

## Question 9

Describe the patch-instructions pass.

Whe you find an instruction with both args as memory locations:

`___q a1, a2` (where a1 and a2 are memory locations)

Produce:
```
movq a1, %rax
___q %rax, a2
```

----

# Register allocation

*Register allocation* is the process of assigning register locations to program variables.

## Question 1

Why is register allocation important?

- Registers are way faster than RAM
- We want ot put variables in registers(instead of RAM) as much as possible, to take your

## Question 2

Consider the program:

```
x = 5
y = x + 6
y + 7
```

Here is the equivalent in x86 assembly, using the stack to hold variable values.

```
...
start:
  movq $5, -8(%rbp)
  movq -8(%rbp), %rax
  movq %rax, -16(%rbp)
  addq $6, -16(%rbp)
  movq -16(%rbp), %rax
  movq %rax, -24(%rbp)
  addq $7, -24(%rbp)
  movq -24(%rbp), %rax
  jmp conclusion
...
```

Convert this program into one that only uses registers.

- 8: r8
- 16: r9
- 24: r10

```
...
start:
  movq $5, %r8
  movq %r8, %rax
  movq %rax, %r9
  addq $6, %r9
  movq %r9, %rax
  movq %rax, %r10
  addq $7, %r10
  movq %r10, %rax
  jmp conclusion
...
```

## Question 3

How many registers are *actually needed* to represent this program?

```
...
start:
  movq $5, %rax
  addq $6, %rax
  addq $7, %rax
  jmp conclusion
...
```
We can actually do the whole program with 1 register

## Question 4

Why is *optimal* register allocation hard?

- It's hard to understand when it's ok to re-user a register
- Theory: there exists a reduction from graph coloring tro optimal register allocation, implying that optimal register allocation is NP-complete.

## Note

Our solution to register allocation will involve three parts:

1. [*Liveness analysis*](https://en.wikipedia.org/wiki/Live_variable_analysis), to understand when a variable's value is no longer needed
2. Building an *interference graph* with edges between variables that cannot share a register
3. Allocating registers to variables by *coloring the interference graph* ([graph coloring](https://en.wikipedia.org/wiki/Graph_coloring) is NP-hard)

----

# Liveness Analysis

Our first pass will perform a *liveness analysis* on the program.

## Question 5

Here is the pseudo-x86 equivalent of the program from earlier, written in x86 assembly syntax:

```
start:
 movq $5, x
 movq x, y
 addq $6, y
 movq y, r
 addq $7, r
 movq r, %rax
```

In the table below, annotate each instruction with its *live-after set*.

Use the following rule to determine the live-after set for instruction $k$:

\begin{align}
L_{after}(k) = (L_{after}(k+1) − W (k+1)) \cup R(k+1)
\end{align}

where $W$ means the variables written by the instruction, and $R$ means the variables read by an instruction. See section 3.2 in the textbook for more.

```
start:
	movq $5, x   {x}
	movq x, y    {y}
	addq $6, y   {y}
	movq y, r    {r}
	addq $7, r   {r}
	movq r, %rax {}
```

## Question 6

Describe the live-after pass in the compiler.

- For each block in the program:
    - Process the instructions in reverse order (starting at the bottom)
    - Start with the empty live-after set for the last instruction
    - Maintain a variable containing the live-after set of the last-processed instruction
    - Build the live-after set for the current instruction, using thg rule above
        - You will need to write functions that tell you what variables are written and read by an instruction
    - Add the new live-after set to the list of live-after sets
    - When finished, reverse the list and return it

----

# Interference Graph

Our second pass will build an *interference graph* using the live-after sets computed earlier. The interference graph is an undirected graph with a node for each program variable, and an edge between each pair of variables which may *not* share a home.

## Question 7

Using the instructions and live-after sets from question 5, draw the interference graph.

Use the following principle: *the writes performed by an instruction must not overwrite something in a live location*.

YOUR SOLUTION HERE

## Question 8

Using the instructions and live-after sets below, draw the interference graph.

| Instruction    | Live-After Set |
| :------------- | :------------- |
| `start:`       | {} |
| `movq $1, a`   | {a}|
| `movq $2, b`   | {a, b} |
| `movq $3, c`   | {a, b, c} |
| `movq a, t`    | {b, c, t} |
| `addq b, t`    | {c, t} |
| `movq t, r`    | {c, r} |
| `addq c, r`    | {r} |
| `movq r, %rax` | {}  |


YOUR SOLUTION HERE

## Question 9

What variables in the above graph can share a register? What variables can't?

YOUR SOLUTION HERE